In [1]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import *
from pyspark.sql.types import *
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)


Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Current idle_timeout is 2800 minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 3.0
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 5
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::343278755431:role/GlueRole
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: cf747cfa-d67c-4c3e-b7e3-f3614af26118
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session cf747cfa-d67c-4c3e-b7e

In [ ]:
args = getResolvedOptions(sys.argv, ['input_file','output_file'])
input_file = args['input_file']
output_file = args['output_file']

In [3]:
hits_df=spark.read.format("csv").option("delimiter","\t").option("header", True).load(input_file)

def find_top_ref(df, hit_time_gmt, user_agent, ip, page_url):
    topmost = df.filter((col("hit_time_gmt") <= hit_time_gmt) & (col("user_agent") == user_agent) & (col("ip") == ip) & (col("page_url") == page_url)).select("hit_time_gmt", "user_agent", "ip", "page_url", "referrer")
    referrer_list = hits_df.filter((col("hit_time_gmt") <= hit_time_gmt) & (col("user_agent") == user_agent) & (col("ip") == ip) & (col("page_url") == page_url)).select("referrer").collect()
    referrer=""
    if len(referrer_list)>0:
        if len(referrer_list[0])>0:
            referrer=referrer_list[0][0]
    if referrer is not None:
        if 'esshopzilla' not in referrer:
            return referrer
        else:
            return find_top_ref(df,hit_time_gmt, user_agent, ip, referrer)
    return referrer

In [5]:
order_df = hits_df.filter("pagename=='Order Complete'")
result_dict = []
for order in order_df.collect():
    result=find_top_ref(hits_df, order['hit_time_gmt'], order['user_agent'], order['ip'], order['page_url'])
    if result and 'esshopzilla' not in result:
        url = result.split(".com")[0].split(".")[-1] + ".com"
        keyword = result.split("?")[-1].split("q=")[1].split("&")[0]
        revenue=int(order['product_list'].split(";")[2]) * int(order['product_list'].split(";")[3])
        result_dict.append({"Search Engine Domain": url, "Search Keyword":keyword, "Revenue": revenue})
final_df=sc.parallelize(result_dict).toDF()
final_df=final_df.groupBy("Search Engine Domain","Search Keyword").agg(sum("Revenue").alias("Revenue"))

In [7]:
final_df.write.mode("overwrite").format("csv").option("delimiter","\t").option("header", True).save(output_file)